### 3.6 손글씨 숫자 인식

글씨를 추론하는 과정만 진행, 그것이 `Foward Propagation`.

In [2]:
'''
'''
import sys,os
sys.path.append(os.pardir) # 부모 디렉토리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True)

print(x_train.shape)    # (60000, 784)
print(t_train.shape)    # (60000,)
print(x_test.shape)     # (10000, 784)
print(t_test.shape)     # (10000,)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [ ]:
'''
MNIST 이미지를 화면으로 불러오기
'''
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()